In [1]:
!pip install neuralforecast

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.4/249.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.7/196.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.2/866.2 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.7 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import torch
from typing import Optional, Union, Tuple


import torch.nn.functional as F
from torch.distributions import Distribution
from torch.distributions import Bernoulli, Normal, StudentT, Poisson, NegativeBinomial

from torch.distributions import constraints


from neuralforecast import NeuralForecast
from neuralforecast.models import TFT
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.utils import AirPassengers, AirPassengersPanel, AirPassengersStatic
from neuralforecast.losses.numpy import rmse, mape
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from neuralforecast.losses.pytorch import MSE, MAE, MQLoss, DistributionLoss

In [3]:
from numpy.random import seed
from random import randrange

In [4]:
df = pd.read_excel("Stock Indexes Daily Realized Covariance (Neural).xlsx", sheet_name= "Vol (Neural)", index_col="Date",parse_dates=True)
columns=df.columns
df_static=pd.read_excel("Static_df (Indexes).xlsx")
n_series=len(df.columns)

In [5]:
n_inputs = [3,5,10,15,21,42,84]
hidden_size = [50,100,150,200,300,500]
epochs=[50,100,150,250,350,450,550,650,750]
dropout=[0,0.2,0.3,0.4]
n_head=[2,4,5,8,10,20]
attn_dropout=[0,0.2,0.3,0.4]
learning_rate=[0.0005,0.0001,0.00005,0.00001]
losses=[MSE(),MAE(),MQLoss(level=[80, 90]),DistributionLoss(distribution='StudentT', level=[80, 90])]
num_lr_decays=[5,3,2,1]
scaler_type=["robust","standard",'minmax']

# Test

In [6]:
test_length=int(len(df['AU200_AUD'])*0.3)
df1=df.unstack().reset_index(name="Value").rename(columns={"level_0":"unique_id", "Date":"ds", "Value":"y"})

In [7]:
model=TFT(h=1, input_size=15, stat_exog_list=['AU200_AUD', 'FR40_EUR', 'JP225_USD', 'UK100_GBP', 'NAS100_USD',
       'SPX500_USD', 'US2000_USD'],
       hidden_size=500,
       n_head=20, attn_dropout=0.4, dropout=0.2, loss=MSE(),
       max_steps=350, learning_rate=learning_rate[3],
       num_lr_decays=num_lr_decays[3],
       step_size=1,
       scaler_type='minmax',
              random_seed=92551658)
fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
forecasts = fcst.cross_validation(df=df1,val_size=1,static_df=df_static,n_windows=None, test_size=test_length-test_length%1,step_size=1)
forecasts = forecasts.dropna()
Y_hat=forecasts["TFT"].values
Y_true=forecasts["y"].values
RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 92551658
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                    | Type                     | Params | Mode 
-----------------------------------------------------------------------------
0 | loss                    | MSE                      | 0      | train
1 | padder_train            | ConstantPad1d            | 0      | train
2 | scaler                  | TemporalNorm             | 0      | train
3 | embedding               | TFTEmbedding             | 9.0 K  | train
4 | static_encoder          | StaticCovariateEncoder   | 13.3 M | train
5 | temporal_encoder        | TemporalCovariateEncoder |

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=350` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

RMSE: 0.003187778227885341 | QLIKE: 0.03891530788360899 | MAE: 0.0017113023530095108 | MAPE: 0.21308062334906955


/usr/local/lib/python3.10/dist-packages/neuralforecast/core.py:209: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


In [8]:
if "TFT-median" not in list(forecasts.columns.values):
  df2=forecasts.reset_index().set_index("ds")[["TFT","unique_id"]].pivot(columns="unique_id")
else:
  df2=forecasts.reset_index().set_index("ds")[["TFT-median","unique_id"]].pivot(columns="unique_id")

In [9]:
df2.to_excel("TFT forecasts.xlsx")